In [8]:
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from lightgbm.sklearn import LGBMClassifier
import numpy as np
import wandb
from wandb.integration.lightgbm import wandb_callback

In [2]:
data = load_iris()
X = data['data']
y = data['target']
feature_names = data['feature_names']
labels = data['target_names']

In [3]:
wandb.init(project='covid-cf', entity='rhacking')
wandb.config.max_depth = 2
clf = LGBMClassifier(max_depth=wandb.config.max_depth)
cv = KFold(shuffle=True)

wandb: Currently logged in as: rhacking (use `wandb login --relogin` to force relogin)
c:\users\roel\.virtualenvs\diagcovid-ckc2berq\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
test_size = 0.2
val_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=True)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size/(1-test_size))

In [11]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [14]:
clf = LGBMClassifier(max_depth=wandb.config.max_depth, objective='multiclass', num_classes=len(np.unique(y)))
clf.fit(X_train, y_train, callbacks=[wandb_callback()])
clf.predict(X_test)

array([1, 1, 0, 2, 2, 0, 2, 2, 0, 0, 2, 1, 2, 2, 1, 0, 2, 0, 1, 2, 2, 1,
       1, 1, 0, 0, 0, 0, 0, 0])

In [15]:
clf = LGBMClassifier(max_depth=wandb.config.max_depth, objective='multiclass', num_classes=len(np.unique(y)))
clf.fit(X_train, y_train, callbacks=[wandb_callback()])
y_pred = clf.predict(X_test)
y_probas = clf.predict_proba(X_test)
wandb.sklearn.plot_classifier(clf, X_train, X_test, y_train, y_test, y_pred, y_probas, labels, model_name='lgbm', feature_names=feature_names)

wandb: 
wandb: Plotting lgbm.
wandb: Logged feature importances.
wandb: Logged learning curve.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.


ValueError: y contains previously unseen labels: [ 16222 114222]

In [3]:
for fold, (train, test) in enumerate(cv.split(X, y)):
    wandb.init(group=f"fold_{fold}", project='covid-cf', entity='rhacking')
    wandb.config.max_depth = 4
    clf = tree.DecisionTreeClassifier(max_depth=wandb.config.max_depth)
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    
    clf.fit(X_train, y_train)
    score = accuracy_score(y_test, clf.predict(X_test))
    wandb.log({'accuracy': score})

wandb: Currently logged in as: rhacking (use `wandb login --relogin` to force relogin)
c:\users\roel\.virtualenvs\diagcovid-ckc2berq\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


accuracy,1.0
_runtime,2
_timestamp,1617277217
_step,0


accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


accuracy,0.96667
_runtime,1
_timestamp,1617277222
_step,0


accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


accuracy,0.96667
_runtime,1
_timestamp,1617277227
_step,0


accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


accuracy,0.86667
_runtime,2
_timestamp,1617277233
_step,0


accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


In [19]:
import pandas as pd
pd.DataFrame({
    'dataset': {
        'name': 'ictcf', 
        'validation': '5 fold'
    }, 
    'preprocessing': {
        'sfm': 0.7
    }, 
    'model_parameters': {
        'n_estimators': 10, 
        'boosting_type': 'dart'
    }
})

,dataset,preprocessing,model_parameters
name,ictcf,NaN,NaN
validation,5 fold,NaN,NaN
sfm,NaN,0.7,NaN
n_estimators,NaN,NaN,10
boosting_type,NaN,NaN,dart


In [21]:
import pandas as pd
import yaml

In [22]:
with open('../configs/ictcf.yaml', 'r') as f:
    df = pd.io.json.json_normalize(yaml.load(f))

c:\users\roel\.virtualenvs\diagcovid-ckc2berq\lib\site-packages\ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [23]:
df

,dataset.name,dataset.target,dataset.validation.fraction,dataset.test.fraction,preprocessing.imputation.categorical,preprocessing.imputation.numeric,preprocessing.rfe,model_parameters.boosting_type,model_parameters.n_estimators,model_parameters.learning_rate
0,ictcf,pcr,0.1,0.1,0,mean,None,dart,40,0.01


In [20]:
!pip install wandb

  Using cached promise-2.3-py3-none-any.whl
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6488 sha256=0971afc899db7bb769fe55ea7c08e477689f9ee2b7a213230c80a845a0c24505
  Stored in directory: c:\users\roel\appdata\local\pip\cache\wheels\50\ca\fa\8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8784 sha256=8111448b970ceafbd67d78336a14724f3e50225edaa65b4e24ff86fa410a74b4
  Stored in directory: c:\users\roel\appdata\local\pip\cache\wheels\3e\31\09\fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [26]:
!wandb login --help

Usage: wandb login [OPTIONS] [KEY]...

  Login to Weights & Biases

Options:
  --cloud        Login to the cloud instead of local
  --host TEXT    Login to a specific instance of W&B
  --relogin      Force relogin if already logged in.
  --anonymously  Log in anonymously
  --help         Show this message and exit.
